In [8]:
import pandas as pd
import numpy as np
import riskfolio 
import matplotlib.pyplot as plt
import pypfopt as ppo
import sqlite3
import os

In [15]:
os.chdir(os.getcwd())
conn = sqlite3.connect(r"C:\databases\portfolio_quantamental.db")
cur = conn.cursor()

ibov_close = pd.read_csv('./data/ativos_ibov/ivob_close.csv')
ibov_close = ibov_close[(ibov_close['Date'] >= '2011-01-01') & (ibov_close['Date'] <= '2020-12-31')]
ibov_close

,Date,BBAS3.SA,BBDC3.SA,BRAP4.SA,BRFS3.SA,BRKM5.SA,CMIG4.SA,CPFE3.SA,CPLE6.SA,CSNA3.SA,...,GOAU4.SA,ITSA4.SA,PETR3.SA,PETR4.SA,SBSP3.SA,TRPL4.SA,USIM5.SA,VALE3.SA,VIVT3.SA,WEGE3.SA
2800,2011-01-03 00:00:00,6.145706,5.161611,4.450233,23.348795,11.365026,2.060042,9.501762,1.736550,12.842748,...,14.462828,2.885917,9.282959,7.498518,10.485193,5.678426,14.745590,25.167242,13.500789,2.473393
2801,2011-01-04 00:00:00,6.171153,5.178123,4.528867,23.581078,11.583906,2.065300,9.529086,1.734465,13.195567,...,14.857988,2.939588,9.209431,7.470745,10.411170,5.598378,15.329878,25.688520,13.412082,2.430632
2802,2011-01-05 00:00:00,6.259226,5.217802,4.690114,24.088663,11.746664,2.080313,9.572347,1.769905,13.524871,...,15.052935,2.976937,9.384058,7.559616,10.623690,5.703441,15.683525,26.152370,13.483036,2.397997
2803,2011-01-06 00:00:00,6.204424,5.092812,4.638357,24.217709,11.606357,2.067545,9.581453,1.750726,13.581325,...,15.021322,2.913224,9.328910,7.529068,10.454151,5.673423,15.568205,25.816633,13.571751,2.372117
2804,2011-01-07 00:00:00,6.057632,5.053132,4.609489,24.501608,11.280838,2.120098,9.654319,1.751143,13.665998,...,15.063467,2.827542,9.191049,7.423533,10.506683,5.666420,15.837285,25.622252,14.015270,2.375493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5276,2020-12-22 00:00:00,14.206707,16.267717,15.720858,21.151329,18.821213,4.433801,21.153643,4.889979,21.535715,...,6.779614,7.435336,9.775627,9.438392,40.334232,20.872669,11.284577,58.565777,35.310616,34.280697
5277,2020-12-23 00:00:00,14.440460,16.615410,15.776933,21.313152,18.959902,4.539444,21.021679,5.037361,21.926497,...,6.885349,7.611438,9.986312,9.670202,41.854881,21.069719,11.809629,58.848701,35.388412,34.186558
5278,2020-12-28 00:00:00,14.599999,16.755821,15.764742,21.370268,18.788578,4.648288,21.252613,5.154997,22.054394,...,6.897791,7.696225,10.060050,9.749777,42.383400,21.213722,11.680385,58.815006,35.707359,35.542336
5279,2020-12-29 00:00:00,14.514665,16.754141,15.618459,21.417862,19.082277,4.632283,21.331793,5.127523,23.127272,...,7.028407,7.741881,10.067070,9.780914,41.483994,21.501726,12.181204,58.653339,35.838055,35.377571
